In [36]:
! wget https://raw.githubusercontent.com/vsubbian/WindowSHAP/main/windowshap.py -P utils/

--2024-10-25 13:03:52--  https://raw.githubusercontent.com/vsubbian/WindowSHAP/main/windowshap.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22720 (22K) [text/plain]
Saving to: ‘utils/windowshap.py’

windowshap.py       100%[===================>]  22.19K  --.-KB/s    in 0.003s  

2024-10-25 13:03:52 (6.82 MB/s) - ‘utils/windowshap.py’ saved [22720/22720]



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
ls

anchor_explanation_cache_checkpoint.pkl
anchor_explanations_coverage.pkl
auxiliary_mushrooms.py
best_model_seq_contrastive.keras
dice_explanation_cache_checkpoint.pkl
dice_explanation_cache_checkpoint_test.pkl
dice_explanations_coverage.pkl
dummy_auc_palm_scale_100_window1.pkl
dummy_auc_palm_scale_100_window2.pkl
dummy_auc_palm_scale_100_window3.pkl
dummy_auc_palm_scale_100_window4.pkl
dummy_auc_palm_scale_100_window5.pkl
dummy_auc_palm_scale_100_window6.pkl
dummy_auc_palm_scale_100_window7.pkl
dummy_prop_perturbation_scale_10_acc.pkl
dummy_prop_perturbation_scale_10_acc_test_fin.pkl
dummy_prop_perturbation_scale_10_acc_train_fin.pkl
extracted_data/
fast/
formatted_table_image.png
lime_auc_palm_scale_100_w0.pkl
lime_auc_palm_scale_100_window1.pkl
lime_auc_palm_scale_100_window2.pkl
lime_auc_palm_scale_100_window3.pkl
lime_auc_palm_scale_100_window4.pkl
lime_auc_palm_scale_100_window5.pkl
lime_explanation_cache_checkpoint.pkl
lime_explanations.pkl
lime_explanations_v02.pkl_window1
lime_

In [3]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 0 - first gpu, 1 - second, "0,1" - both gpu, first used

import gc
import numpy as np
import pandas as pd
# import io
import pickle
import shap
import sys
import tensorflow as tf
import tensorflow.python.keras as keras  # import keras
import warnings
from aeon.datasets import load_classification
from keras.layers import ConvLSTM1D
from keras.utils import to_categorical
from lime import lime_tabular
from numpy import mean, std
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.python.keras.initializers import glorot_uniform
from tensorflow.python.util import deprecation
from typing import List, Tuple, Any
from utils.windowshap import SlidingWindowSHAP

print(tf.__version__)  # print(tf.keras.__version__)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
2024-10-30 18:08:24.973556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-30 18:08:25.633488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.1


In [4]:
tf.compat.v1.disable_v2_behavior()

warnings.filterwarnings('ignore')
ops.logging.set_verbosity(ops.logging.ERROR)
tf.get_logger().setLevel('FATAL')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
deprecation._PRINT_DEPRECATION_WARNINGS = False
os.environ['TF_ENABLE_DEPRECATION_WARNINGS'] = '0'
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

gpus = tf.config.list_physical_devices("GPU")
print(gpus)
gpu_to_limit = gpus[0]  # or 1, watch out for CUDA_VISIBLE_DEVICES
print(gpu_to_limit)

Instructions for updating:
non-resource variables are not supported in the long term
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [5]:
gpu_mem_mb = 24564  # tyle mamy
tf.config.set_logical_device_configuration(
    gpu_to_limit, [tf.config.LogicalDeviceConfiguration(memory_limit=gpu_mem_mb // 2)]
)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

data_path = "shared/UCI-Benchmark/"
batch_size = 64

Num GPUs Available:  1


# Real Data example

In [6]:
# convlstm model
def load_dataset_aeon(dsname):
    X, y, meta_data = load_classification(dsname)
    X = np.moveaxis(X, 1, 2)

    le = LabelEncoder()
    y = le.fit_transform(y)
    trainX, testX, trainy, testy = train_test_split(X, y)

    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    return trainX, trainy, testX, testy


# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


# run an experiment
def train_model(trainX, trainy, testX, testy):
    return 1.0, keras.models.Sequential([])


def train_lstmconv_model(trainX, trainy, testX, testy):
    return 1.0, keras.models.Sequential([])


def evaluate_model(dataset, repeats=1):
    # load data
    trainX, trainy, testX, testy = load_dataset_aeon(dataset)
    #fill missing with zeros
    trainX = np.nan_to_num(trainX, nan=0)
    testX = np.nan_to_num(testX, nan=0)

    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX.reshape(-1, trainX.shape[-1])).reshape(trainX.shape)
    testX = scaler.transform(testX.reshape(-1, testX.shape[-1])).reshape(testX.shape)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score, model = train_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r + 1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    return model, trainX, trainy, testX, testy, score


# run an experiment
def evaluate_lstmconv_model(dataset, repeats=1):
    # load data
    trainX, trainy, testX, testy = load_dataset_aeon(dataset)

    #fill missing with zeros
    trainX = np.nan_to_num(trainX, nan=0)
    testX = np.nan_to_num(testX, nan=0)

    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX.reshape(-1, trainX.shape[-1])).reshape(trainX.shape)
    testX = scaler.transform(testX.reshape(-1, testX.shape[-1])).reshape(testX.shape)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score, model = train_lstmconv_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r + 1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    return model, trainX, trainy, testX, testy, score


## save & load bundle

In [7]:
def save_bundle(model, trainX, trainy, testX, testy, svtr, svts, dsname,
                dir='./results', explain_prefix="sv", only_explain=False):
    if not os.path.isdir(f'{dir}/{dsname}'):
        os.makedirs(f'{dir}/{dsname}')

    if not only_explain:
        # model.save(f'{dir}/{dsname}/model.h5', save_format='h5')
        model.save(f'{dir}/{dsname}/model_tf', save_format='tf')

        pickle.dump(trainX, open(f'{dir}/{dsname}/trainX.pickle', 'wb'))
        pickle.dump(trainy, open(f'{dir}/{dsname}/trainy.pickle', 'wb'))
        pickle.dump(testX, open(f'{dir}/{dsname}/testX.pickle', 'wb'))
        pickle.dump(testy, open(f'{dir}/{dsname}/testy.pickle', 'wb'))

    pickle.dump(svtr, open(f'{dir}/{dsname}/{explain_prefix}tr.pickle', 'wb'))
    pickle.dump(svts, open(f'{dir}/{dsname}/{explain_prefix}ts.pickle', 'wb'))


def load_bundle(dsname, dir='./results', explain_prefix="sv", only_explain=False):
    model, trainX, trainy, testX, testy = None, None, None, None, None,
    if not only_explain:
        tf_model_path = f'{dir}/{dsname}/model_tf'
        h5_model_path = f'{dir}/{dsname}/model.h5'

        if os.path.exists(tf_model_path):
            print("Loading model in tf format...")
            model = tf.keras.models.load_model(tf_model_path)
        elif os.path.exists(h5_model_path):
            print("Loading model in h5 format...")
            model = tf.keras.models.load_model(h5_model_path,
                                               custom_objects={'GlorotUniform': glorot_uniform,
                                                               'ConvLSTM1D': ConvLSTM1D})
        else:
            raise FileNotFoundError(
                f'Nie znaleziono modelu w formatach SavedModel ani H5 dla {dsname} w katalogu {dir}.')
        # print(model.summary())

        trainX = pickle.load(open(f'{dir}/{dsname}/trainX.pickle', 'rb'))
        trainy = pickle.load(open(f'{dir}/{dsname}/trainy.pickle', 'rb'))
        testX = pickle.load(open(f'{dir}/{dsname}/testX.pickle', 'rb'))
        testy = pickle.load(open(f'{dir}/{dsname}/testy.pickle', 'rb'))

    svtr = pickle.load(open(f'{dir}/{dsname}/{explain_prefix}tr.pickle', 'rb'))
    svts = pickle.load(open(f'{dir}/{dsname}/{explain_prefix}ts.pickle', 'rb'))

    return model, trainX, trainy, testX, testy, svtr, svts

In [8]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stderr.close()
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr


# @tf.function
def evaluate_model(model, testX, testy, batch_size, steps):
    # f = io.StringIO()
    # with redirect_stdout(f), redirect_stderr(f):
    # sys.stderr = open(os.devnull, 'w')
    if steps is not None:
        result = model.evaluate(testX, testy, batch_size=batch_size, verbose=0, steps=steps)
    else:
        result = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    # sys.stderr = sys.__stderr__
    return result

# Univariate

In [9]:
#list directoies in  folder
datasets_uni = [f.name for f in os.scandir(data_path + 'ds/univariate') if f.is_dir() and not f.name.startswith('.')]
print(datasets_uni)

['DodgerLoopGame', 'ProximalPhalanxTW', 'DodgerLoopDay', 'ECGFiveDays', 'UMD', 'Plane', 'ECG200', 'TwoPatterns', 'GunPointAgeSpan', 'DiatomSizeReduction', 'UWaveGestureLibraryAll', 'MedicalImages', 'Meat', 'Trace', 'MiddlePhalanxOutlineAgeGroup', 'Chinatown', 'DistalPhalanxOutlineAgeGroup', 'WormsTwoClass', 'DistalPhalanxOutlineCorrect', 'Strawberry', 'OliveOil', 'UWaveGestureLibraryX', 'SmoothSubspace', 'Fungi', 'ElectricDevices', 'SwedishLeaf', 'CricketX', 'ECG5000', 'PhalangesOutlinesCorrect', 'FaceFour', 'SyntheticControl', 'FordA', 'PowerCons', 'BeetleFly', 'GunPointMaleVersusFemale', 'Yoga', 'Herring', 'Crop', 'RefrigerationDevices', 'Worms', 'OSULeaf', 'ItalyPowerDemand', 'GunPoint', 'CBF', 'Symbols', 'ToeSegmentation2', 'TwoLeadECG', 'SmallKitchenAppliances', 'ShapesAll', 'ScreenType', 'Computers', 'CricketY', 'SonyAIBORobotSurface2', 'Wafer', 'MiddlePhalanxOutlineCorrect', 'Lightning2', 'UWaveGestureLibraryY', 'Wine', 'ProximalPhalanxOutlineCorrect', 'ShapeletSim', 'CricketZ',

In [32]:
for p in datasets_uni:
    print(f'Loading... {p}')
    ##trainsvo is shap calculated for train, testsv is shap calculated for test (NOTE: these are absolute values of shap)
    model, trainXo, trainyo, testX, testy, trainsvo, testsv = load_bundle(p, dir=data_path + 'ds/univariate')
    # with HiddenPrints():
    _, score = evaluate_model(model, testX, testy, batch_size=batch_size, steps=None)
    score = score * 100.0
    print('>Accuracy: %.3f' % (score))

    K.clear_session()
    del model, trainXo, trainyo, testX, testy, trainsvo, testsv
    gc.collect()

    break


Loading... DodgerLoopGame


2024-10-25 10:46:38.798457: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_2/kernel/Assign' id:1710 op device:{requested: '', assigned: ''} def:{{{node dense_2/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_2/kernel, dense_2/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-10-25 10:46:38.995269: W tensorflow/c/c_api.cc:304] Operation '{name:'module_wrapper_2/conv_lstm1d/kernel/v/Assign' id:1943 op device:{requested: '', assigned: ''} def:{{{node module_wrapper_2/conv_lstm1d/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](module_wrapper_2/conv_lstm1d/kernel/v, module_wrapper_2/conv_lstm1d/kernel/v/Initializer/zeros)}}' was changed by setting attr

>Accuracy: 87.500


# Multivariate

In [11]:
#list directoies in  folder
datasets_multi = [f.name for f in os.scandir(data_path + '/ds/multivariate') if
                  f.is_dir() and not f.name.startswith('.')]

In [7]:
for p in datasets_multi:
    print(f'Loading... {p}')
    model, trainXo, trainyo, testX, testy, trainsvo, testsv = load_bundle(p, dir=data_path + 'ds/multivariate')
    # steps = len(testX) // batch_size
    # if len(testX) % batch_size != 0:
    #     steps += 1
    _, score = evaluate_model(model, testX, testy, batch_size, None)
    score = score * 100.0
    print('>Accuracy: %.3f' % (score))

    # K.clear_session()
    # del model, trainXo, trainyo, testX, testy, trainsvo, testsv
    # gc.collect()

    break

Loading... ArticularyWordRecognition


2024-10-29 21:04:51.045931: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5323 MB memory:  -> device: 0, name: NVIDIA RTX A5500, pci bus id: 0000:51:00.0, compute capability: 8.6
2024-10-29 21:04:51.046743: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 12282 MB memory:  -> device: 1, name: NVIDIA RTX A5500, pci bus id: 0000:9c:00.0, compute capability: 8.6
2024-10-29 21:04:51.059459: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2024-10-29 21:04:51.156898: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_1/kernel/Assign' id:683 op device:{requested: '', assigned: ''} def:{{{node dense_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_1/kernel, dense_1/kernel/Initializer/random_uniform)}}' was changed by se

>Accuracy: 96.528


In [8]:
dir(model)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 3, 48, 9)          0         
                                                                 
 conv_lstm1d (ConvLSTM1D)    (None, 3, 48, 64)         168448    
                                                                 
 conv_lstm1d_1 (ConvLSTM1D)  (None, 3, 48, 32)         110720    
                                                                 
 dropout (Dropout)           (None, 3, 48, 32)         0         
                                                                 
 embedding (Flatten)         (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 100)               460900    
                                                                 
 dense_1 (Dense)             (None, 25)                2

In [9]:
print(trainXo.shape)
print(trainsvo.shape)

(431, 144, 9)
(431, 144, 9)


# Explainers

In [12]:
def prepare_data(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path, header=None, sep='|')
    df = df[0].str.split(',', expand=True).dropna(axis=1)
    df.columns = df.iloc[0]
    return df[1:]


def load_background_data(trainX: np.ndarray, bg_size: int) -> np.ndarray:
    indexes = np.arange(len(trainX))
    np.random.shuffle(indexes)
    maxid = min(bg_size, len(trainX))
    return trainX[indexes[:maxid]]


def compute_window_shap(model: Any, trainX: np.ndarray, testX: np.ndarray, window_len: int, stride: int,
                        bg_data: np.ndarray, output_shape: int) -> Tuple[np.ndarray, np.ndarray]:
    sv_ts = np.zeros((len(testX), testX.shape[1], testX.shape[2]))
    sv_tr = np.zeros((len(trainX), trainX.shape[1], trainX.shape[2]))

    for i in range(len(testX)):
        gtw = SlidingWindowSHAP(model, stride, window_len, bg_data, testX[i:i + 1], model_type='lstm')
        sv_ts[i, :, :] = gtw.shap_values(num_output=output_shape)

    for i in range(len(trainX)):
        gtw = SlidingWindowSHAP(model, stride, window_len, bg_data, trainX[i:i + 1], model_type='lstm')
        sv_tr[i, :, :] = gtw.shap_values(num_output=output_shape)

    return sv_tr, sv_ts


def compute_deep_shap(model: Any, trainX: np.ndarray, testX: np.ndarray, bg_data: np.ndarray, absshap: bool) -> Tuple[
    np.ndarray, np.ndarray]:
    explainer = shap.DeepExplainer(model, bg_data)
    shap_values_ts = explainer.shap_values(testX, check_additivity=False)
    shap_values_tr = explainer.shap_values(trainX, check_additivity=False)

    if absshap:
        return abs(np.array(shap_values_tr)).mean(axis=0), abs(np.array(shap_values_ts)).mean(axis=0)

    indexer_ts = np.argmax(model.predict(testX), axis=1)
    sv_ts = np.concatenate([shap_values_ts[indexer_ts[i]][i, :] for i in range(len(testX))])

    indexer_tr = np.argmax(model.predict(trainX), axis=1)
    sv_tr = np.concatenate([shap_values_tr[indexer_tr[i]][i, :] for i in range(len(trainX))])

    return sv_tr, sv_ts


# , num_features: int = 10
def compute_deep_lime(model: Any, trainX: np.ndarray, testX: np.ndarray) -> Tuple[
    np.ndarray, np.ndarray]:
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]

    trainX_reshaped = trainX.reshape(-1, n_timesteps * n_features)
    testX_reshaped = testX.reshape(-1, n_timesteps * n_features)

    nan_count_train_X = np.isnan(trainX_reshaped).sum()
    nan_count_test_X = np.isnan(testX_reshaped).sum()
    total_train_elements = trainX_reshaped.size
    total_test_elements = testX_reshaped.size

    if nan_count_train_X > 0 or nan_count_test_X > 0:
        nan_percentage_train_X = (nan_count_train_X / total_train_elements) * 100
        nan_percentage_test_X = (nan_count_test_X / total_test_elements) * 100
        print(f"Warning: Found NaN values. Train NaNs: {nan_count_train_X} ({nan_percentage_train_X:.2f}%), "
              f"Test NaNs: {nan_count_test_X} ({nan_percentage_test_X:.2f}%)")

        most_frequent_train_X = np.nanmedian(trainX_reshaped)
        # most_frequent_test = np.nanmedian(testX_reshaped)

        # Calculate the most frequent value across both samples
        combined_data_X = np.concatenate((trainX_reshaped[~np.isnan(trainX_reshaped)],
                                          testX_reshaped[~np.isnan(testX_reshaped)]))
        most_frequent_value_X = np.bincount(combined_data_X.astype(int)).argmax()

        print(
            f"Replacing NaN values with the most frequent value in train: {most_frequent_train_X}, while most frequent overall is  {most_frequent_value_X}")

        trainX_reshaped = np.where(np.isnan(trainX_reshaped), most_frequent_train_X, trainX_reshaped)
        testX_reshaped = np.where(np.isnan(testX_reshaped), most_frequent_train_X, testX_reshaped)

    train_predictions = model.predict(trainX.reshape(-1, n_timesteps, n_features))
    test_predictions = model.predict(testX.reshape(-1, n_timesteps, n_features))

    print("# of non-Nan predictions for train", np.count_nonzero(~np.isnan(train_predictions)),
          "of", train_predictions.size)
    print("# of non-Nan predictions for test", np.count_nonzero(~np.isnan(test_predictions)),
          "of", test_predictions.size)

    nan_threshold_y = 1.0  # percent
    nan_count_train_y = np.isnan(train_predictions).sum()
    nan_count_test_y = np.isnan(test_predictions).sum()

    nan_percentage_train_y = (nan_count_train_y / train_predictions.size) * 100
    nan_percentage_test_y = (nan_count_test_y / test_predictions.size) * 100

    if nan_percentage_train_y > nan_threshold_y or nan_percentage_test_y > nan_threshold_y:
        print(f"Error: NaN percentage in predictions exceeded threshold. "
              f"Train NaNs: {nan_count_train_y} ({nan_percentage_train_y:.2f}%), "
              f"Test NaNs: {nan_count_test_y} ({nan_percentage_test_y:.2f}%).")
        print("Stopping LIME calculations and returning empty results.")

        return np.empty((len(trainX), n_timesteps, n_features)), np.empty(
            (len(testX), n_timesteps, n_features))  # return np.array([]), np.array([])
    elif nan_percentage_train_y > 0.0 or nan_percentage_test_y > 0.0:
        print(f"Warning: There are NaN's in predictions. "
              f"Train NaNs: {nan_count_train_y} ({nan_percentage_train_y:.2f}%), "
              f"Test NaNs: {nan_count_test_y} ({nan_percentage_test_y:.2f}%).")

    most_frequent_value_y = np.nanmedian(
        train_predictions)  # np.bincount(train_predictions[~np.isnan(train_predictions)].astype(int)).argmax()

    def safe_predict_fn(x):
        prediction = model.predict(x.reshape(-1, n_timesteps, n_features))

        # Check for NaNs in the prediction
        if np.isnan(prediction).sum() > 0:
            nan_count_prediction = np.isnan(prediction).sum()
            nan_percentage_prediction = (nan_count_prediction / prediction.size) * 100
            # Replace NaNs in the prediction with a default value  
            prediction = np.nan_to_num(prediction, nan=most_frequent_value_y)
            print(f"Warning: Prediction contains NaN values "
                  f"({nan_count_prediction} NaNs, {nan_percentage_prediction:.2f}%). "
                  f"Replacing NaN values in prediction with {most_frequent_value_y}.")

        return prediction

    feature_names = [f'feature_{i}' for i in range(n_timesteps * n_features)]
    class_names = [f'class_{i}' for i in range(model.output_shape[-1])]

    explainer = lime_tabular.LimeTabularExplainer(
        training_data=trainX_reshaped,
        feature_names=feature_names,
        class_names=class_names,
        mode="classification",
        discretize_continuous=True
    )

    # sv_ts = np.zeros((len(testX), testX.shape[1], testX.shape[2]))
    # sv_tr = np.zeros((len(trainX), trainX.shape[1], trainX.shape[2]))
    sv_ts = []
    sv_tr = []

    for i in range(len(testX)):
        # for i in tqdm(range(len(testX)), desc="Processing test data"):
        # print(f"[{i}] testX: start")
        explanation = explainer.explain_instance(
            data_row=testX_reshaped[i],
            predict_fn=safe_predict_fn,
            num_features=n_timesteps * n_features
        )
        weights = np.array([feature[1] for feature in explanation.as_list()])
        shaped_weights = weights.reshape((n_timesteps, n_features))
        sv_ts.append(shaped_weights)

    for i in range(len(trainX)):
        # for i in tqdm(range(len(trainX)), desc="Processing train data"):
        explanation = explainer.explain_instance(
            data_row=trainX_reshaped[i],
            predict_fn=safe_predict_fn,  # lambda x: model.predict(x.reshape(-1, n_timesteps, n_features))
            num_features=n_timesteps * n_features
        )
        weights = np.array([feature[1] for feature in explanation.as_list()])
        shaped_weights = weights.reshape((n_timesteps, n_features))
        sv_tr.append(shaped_weights)

    return np.array(sv_tr), np.array(sv_ts)


def process_dataset(p: str, model: Any, trainX: np.ndarray, trainy: np.ndarray, testX: np.ndarray, testy: np.ndarray,
                    bg_size: int, window_len: int, stride: int, explanation_method: str, absshap: bool,
                    shap_enabled: bool, lime_enabled: bool) -> Tuple[
    List[Any], Tuple[np.ndarray], List[Any], Tuple[np.ndarray]]:
    print(f"Processing... {p}")
    bg_data = load_background_data(trainX, bg_size)

    sv_tr, sv_ts = None, None
    lime_tr, lime_ts = None, None

    if shap_enabled:
        if explanation_method == 'window':
            print("SHAP: window")
            sv_tr, sv_ts = compute_window_shap(model, trainX, testX, window_len, stride, bg_data, trainy.shape[1])
        else:
            print("SHAP: deep")
            sv_tr, sv_ts = compute_deep_shap(model, trainX, testX, bg_data, absshap)

    if lime_enabled:
        print("LIME: deep")
        lime_tr, lime_ts = compute_deep_lime(model, trainX, testX)

    return sv_tr, sv_ts, lime_tr, lime_ts

In [13]:
bg_size = 1000
window_len = 23
stride = 10
explanation_method = 'deep'  # or 'window' if needed for SHAP window-based explanations
absshap = True
shap_enabled = False  # SHAP disabled
lime_enabled = True  # LIME enabled
explain_prefix = "lv"

print(data_path, batch_size, bg_size, window_len, stride, explanation_method, absshap, shap_enabled, lime_enabled,
      explain_prefix)

summary = []

shared/UCI-Benchmark/ 64 1000 23 10 deep True False True lv


In [ ]:
directory = 'univariate'

for p in datasets_uni:
    print(f'Loading... {p}')

    explain_tr_path = os.path.join(data_path, 'ds', directory, p, f'{explain_prefix}tr.pickle')
    explain_ts_path = os.path.join(data_path, 'ds', directory, p, f'{explain_prefix}ts.pickle')
    if os.path.exists(explain_tr_path) and os.path.exists(explain_ts_path):
        if os.path.getsize(explain_tr_path) > 1024 and os.path.getsize(explain_ts_path) > 1024:
            print(f"Omitting {p}, explainers {directory} SHAP:{shap_enabled} LIME:{lime_enabled} already calculated.")
            continue

    model, trainX, trainy, testX, testy, _, _ = load_bundle(p, dir=os.path.join(data_path, 'ds', directory))

    sv_tr, sv_ts, lime_tr, lime_ts = process_dataset(p, model, trainX, trainy, testX, testy,
                                                     bg_size, window_len, stride, explanation_method, absshap,
                                                     shap_enabled, lime_enabled)

    _, score = evaluate_model(model, testX, testy, batch_size, None)
    score = score * 100.0
    print('>Accuracy: %.3f' % (score))

    print(f'Saving bundle... {p},  SHAP: {shap_enabled}, LIME: {lime_enabled}')
    save_bundle(model, trainX, trainy, testX, testy, lime_tr, lime_ts, p,
                dir=os.path.join(data_path, 'ds', directory), explain_prefix=explain_prefix, only_explain=True)
    summary.append([directory, p, score, 'OK'])

    tf.keras.backend.clear_session()
    del model, trainX, trainy, testX, testy, sv_tr, sv_ts, lime_tr, lime_ts
    gc.collect()

    # break

In [ ]:
directory = 'multivariate'

for p in datasets_multi:
    print(f'Loading... {p}')

    explain_tr_path = os.path.join(data_path, 'ds', directory, p, f'{explain_prefix}tr.pickle')
    explain_ts_path = os.path.join(data_path, 'ds', directory, p, f'{explain_prefix}ts.pickle')
    if os.path.exists(explain_tr_path) and os.path.exists(explain_ts_path):
        if os.path.getsize(explain_tr_path) > 1024 and os.path.getsize(explain_ts_path) > 1024:
            print(f"Omitting {p}, explainers {directory} SHAP:{shap_enabled} LIME:{lime_enabled} already calculated.")
            continue

    model, trainX, trainy, testX, testy, _, _ = load_bundle(p, dir=os.path.join(data_path, 'ds', directory))

    sv_tr, sv_ts, lime_tr, lime_ts = process_dataset(p, model, trainX, trainy, testX, testy,
                                                     bg_size, window_len, stride, explanation_method, absshap,
                                                     shap_enabled, lime_enabled)

    _, score = evaluate_model(model, testX, testy, batch_size, None)
    score = score * 100.0
    print('>Accuracy: %.3f' % score)

    print(f'Saving bundle... {p}, SHAP: {shap_enabled}, LIME: {lime_enabled}')
    save_bundle(model, trainX, trainy, testX, testy, lime_tr, lime_ts, p,
                dir=os.path.join(data_path, 'ds', directory), explain_prefix=explain_prefix, only_explain=True)
    summary.append([directory, p, score, 'OK'])

    tf.keras.backend.clear_session()
    del model, trainX, trainy, testX, testy, sv_tr, sv_ts, lime_tr, lime_ts
    gc.collect()

    # break

Loading... ArticularyWordRecognition
Omitting ArticularyWordRecognition, explainers multivariate SHAP:False LIME:True already calculated.
Loading... Handwriting
Omitting Handwriting, explainers multivariate SHAP:False LIME:True already calculated.
Loading... BasicMotions
Omitting BasicMotions, explainers multivariate SHAP:False LIME:True already calculated.
Loading... HandMovementDirection
Omitting HandMovementDirection, explainers multivariate SHAP:False LIME:True already calculated.
Loading... Heartbeat
Omitting Heartbeat, explainers multivariate SHAP:False LIME:True already calculated.
Loading... FaceDetection


2024-10-30 18:09:13.673803: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12282 MB memory:  -> device: 0, name: NVIDIA RTX A5500, pci bus id: 0000:9c:00.0, compute capability: 8.6
2024-10-30 18:09:13.687022: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2024-10-30 18:09:13.721764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:753] failed to allocate 11.99GiB (12878610432 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-10-30 18:09:13.870115: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_1/kernel/Assign' id:683 op device:{requested: '', assigned: ''} def:{{{node dense_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_1/kernel, dense_1/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutatio

Processing... FaceDetection
LIME: deep


In [72]:
# model
# trainX.shape
# trainy.shape

In [75]:
# save_bundle(model, trainX, trainy, testX, testy, lime_tr, lime_ts, p,
#             dir=data_path + '/ds/multivariate', explain_prefix="lv", only_explain=True)

In [ ]:
pd.DataFrame(summary)

In [16]:
pd.DataFrame(summary, columns=['dataset', 'score', 'status']).to_csv(data_path + '/ds/summary-multi-1.csv',
                                                                     index=False)